# Излечение, преобразование, загрузка

## Импортируем необходимые модули

In [80]:
import pandas as pd
from sqlalchemy import create_engine, text

## Настраиваем подключение к БД

In [81]:
user = "entries_user"
password = "entries_password"
host = "localhost"
port = "5432"
database = "entries_db"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

## intervals_tgt

Основа обработки - итерация по датам (ГГГГ-ММ-ДД) для каждого вхождения full_name, рассмотрение каждой записи, формирование из наиболее близких "Вход" и "Выход" интервала. Рассмотрим обработку аномалий:
1. Ошибки системы - из пар записей с одинаковыми статусами и временной разницей менее минуты выбираем более позднюю, другую отбрасываем.
2. Экстренный вход - если выход был произведён в обход системы, то повторный вход не рассматриваем как действительное начало интервала.
3. Ночные выходы - если последней по времени записью для указанной даты является вход, то ищем самый ранний выход в следующем дне. По условию сотрудник не может пропасть или выйти, минуя систему, и не вернуться, учтённый выход на следующий день помечаем, чтобы избежать его повторного учёта.
4. Перерыв на обед - если для указанной даты несколько пар входов и выходов или пара входов и выходов и вход, то учитываем несколько интервалов для одной даты.
5. Конец учётного периода - если сотрудник остался в ночь, совпадающую с окончанием отчётного периода, то его интервал закрывается датой и временем окончания отчётного периода.

In [82]:
df = pd.read_sql("SELECT * FROM entries_src ORDER BY full_name, event_dt", engine)
df['event_dt'] = pd.to_datetime(df['event_dt'])
df['event_date'] = df['event_dt'].dt.date

report_period_end = df['event_dt'].max().normalize() + pd.Timedelta(hours=23, minutes=59, seconds=59)

results = []
used_indexes = set()

for name, group in df.groupby('full_name'):
    group = group.sort_values('event_dt').reset_index()
    grouped = group.groupby(group['event_dt'].dt.date)

    dates = sorted(grouped.groups.keys())
    i = 0

    while i < len(dates):
        date = dates[i]
        day_records = grouped.get_group(date).copy()

        deduped = []
        prev_row = None
        
        for _, row in day_records.iterrows():
            if row['index'] in used_indexes:
                continue
        
            if prev_row is not None:
                same_status = prev_row['status'] == row['status']
                time_diff = (row['event_dt'] - prev_row['event_dt']).total_seconds()
        
                if same_status and time_diff <= 60:
                    deduped.pop()
        
            deduped.append(row)
            prev_row = row
        
        day_records = pd.DataFrame(deduped)

        j = 0
        
        while j < len(day_records):
            row = day_records.iloc[j]
        
            if row['status'] != 'Вход' or row['index'] in used_indexes:
                j += 1
                continue

            enter_dt = row['event_dt']
            exit_dt = None
            used_indexes.add(row['index'])

            for k in range(j + 1, len(day_records)):
                next_row = day_records.iloc[k]
        
                if next_row['index'] in used_indexes:
                    continue

                if next_row['status'] in ['Выход', 'Доступ запрещён']:
                    exit_dt = next_row['event_dt']
                    used_indexes.add(next_row['index'])
                    j = k + 1
                    break

                elif next_row['status'] == 'Вход':
        
                    if (next_row['event_dt'] - enter_dt).total_seconds() <= 60:
                        enter_dt = next_row['event_dt']
                        used_indexes.add(next_row['index'])
                        continue
                    else:
                        j = k
                        break
            else:
                found = False
        
                if i + 1 < len(dates):
                    next_day_records = grouped.get_group(dates[i + 1])
        
                    for _, next_row in next_day_records.iterrows():
                        if next_row['index'] in used_indexes:
                            continue
        
                        if next_row['status'] in ['Выход', 'Доступ запрещён']:
                            exit_dt = next_row['event_dt']
                            used_indexes.add(next_row['index'])
                            found = True
                            break

                if not found:
                    exit_dt = report_period_end

                j += 1

            results.append({
                'full_name': name,
                'enter_dt': enter_dt,
                'exit_dt': exit_dt
            })

        i += 1


intervals_df = pd.DataFrame(results)
intervals_df = intervals_df.sort_values(['full_name', 'enter_dt']).reset_index(drop=True)
intervals_df['id'] = intervals_df.index + 1

intervals_df[['id', 'full_name', 'enter_dt', 'exit_dt']].to_sql(
    'intervals_tgt',
    engine,
    if_exists='append',
    index=False
)

462

## workdays_tgt

Для каждого сотрудника рассматриваем все возможные даты (ГГГГ-ММ-ДД), для каждой даты в качестве начала рабочего дня считается наименьшее значение начала интервала, а в качестве окончания рабочего дня считается значение окончания интервала, соответствующее наибольшему значению начала интервала, соответствующей рассматриваемой даты.

In [83]:
df = pd.read_sql("SELECT * FROM intervals_tgt ORDER BY full_name, enter_dt", engine)
df['enter_dt'] = pd.to_datetime(df['enter_dt'])
df['exit_dt'] = pd.to_datetime(df['exit_dt'])

df['report_dt'] = df['enter_dt'].dt.date

results = []

for name, group in df.groupby('full_name'):
    group = group.sort_values('enter_dt').reset_index(drop=True)
    
    grouped_by_date = group.groupby('report_dt')

    for date, day_group in grouped_by_date:
        enter_dt = day_group['enter_dt'].min()
        exit_dt = day_group[day_group['enter_dt'] == day_group['enter_dt'].max()]['exit_dt'].iloc[0]

        results.append({
            'full_name': name,
            'report_dt': date,
            'enter_dt': enter_dt,
            'exit_dt': exit_dt
        })

workdays_df = pd.DataFrame(results)

workdays_df['id'] = workdays_df.index + 1

workdays_df[['id', 'full_name', 'report_dt', 'enter_dt', 'exit_dt']].to_sql(
    'workdays_tgt',
    engine,
    if_exists='append',
    index=False
)

318

## aggregated_info_tgt

In [84]:
query_aggregated_info = """
INSERT INTO aggregated_info_tgt (
    full_name,
    month,
    workdays_count,
    on_time_count,
    late_0_15,
    late_15_30,
    late_30_60,
    late_60_plus,
    full_day_count,
    short_day_count,
    avg_worktime
)
SELECT
    full_name,
    TO_CHAR(report_dt, 'YYYY-MM') AS month,
    COUNT(*) AS workdays_count,
    COUNT(*) FILTER (WHERE enter_dt::time <= TIME '09:00:00') AS on_time_count,
    COUNT(*) FILTER (
        WHERE enter_dt::time > TIME '09:00:00'
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 15
    ) AS late_0_15,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 15
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 30
    ) AS late_15_30,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 30
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 60
    ) AS late_30_60,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 60
    ) AS late_60_plus,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600 >= 9
    ) AS full_day_count,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600 < 9
    ) AS short_day_count,
    ROUND(AVG(EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600), 2) AS avg_worktime
FROM workdays_tgt
GROUP BY full_name, TO_CHAR(report_dt, 'YYYY-MM')
ORDER BY full_name, TO_CHAR(report_dt, 'YYYY-MM');
"""

with engine.connect() as conn:
    with conn.begin():
        conn.execute(text(query_aggregated_info))

## Эскпортируем полученные таблицы в .csv

In [85]:
tables = ['entries_src', 'intervals_tgt', 'aggregated_info_tgt']

for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", engine)
    csv_filename = f"data/target/{table}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Таблица {table} сохранена в файл {csv_filename}")

Таблица entries_src сохранена в файл data/target/entries_src.csv
Таблица intervals_tgt сохранена в файл data/target/intervals_tgt.csv
Таблица aggregated_info_tgt сохранена в файл data/target/aggregated_info_tgt.csv
